# IMPORTS

In [ ]:
%run ipynb_setup.ipynb

In [ ]:
%run class_Dataset.ipynb

In [ ]:
%run class_TokenSearch.ipynb

# CLASS DEF

In [ ]:
class ProductSearch():

    def __init__(
        self,
        ) -> None :

        self.dataset      = Dataset() # initialize Wish dataset
        self.token_search = TokenSearch(dataset=self.dataset) # prep token searcher
        
    def query(
        self,
        search_string : str,
        exact_search  : bool = False,
        plot_top_n    : int  = None,
        ) -> pd.DataFrame :
        ########################################
        # step 1 = apply text search
        ########################################
        token_search_results = self.token_search.tokens_found_count(
            search_string = search_string,
            exact_search  = exact_search,
            verbose       = 1,
        )
    
        ########################################
        # step 2 = grab products with most found tokens
        ########################################
        # TODO # HANDLE CASE WHERE EMPTY DF
        max_tokens_matched = token_search_results['tokens_matched'].max()
        most_found_token_results = token_search_results[token_search_results['tokens_matched'] == max_tokens_matched]
        
        ###########################################################
        # plot_top_n results
        ###########################################################
        if (plot_top_n is not None) and len(most_found_token_results)>0:
            self.dataset.get_product_pictures(locs=most_found_token_results.index[:plot_top_n])
            plt.show()

        return most_found_token_results

In [ ]:
ps=ProductSearch()
ps.query('harajuku beach',plot_top_n=5)
#ps.query('dress beach flower dot silt',plot_top_n=5)